In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from pathlib import Path
import os
import sys
import csv

pd.set_option('display.max_columns', None)
rootdir = os.path.join(os.path.abspath(''), 'Assignment2_Company_ETL')

In [9]:
# collect dataframes from files
df_list = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith('.xlsx'):
            # extract file from each file path found by os.walk()
            df = pd.read_excel(os.path.join(root, file), ignore_index=True)
            
            # attach month & country values
            month, country = Path(root).parts[-2:]
            if (month == 'yearly'):
                month = float("NaN")
            df["month"] = pd.DataFrame([month]*len(df))
            df["country"] = pd.DataFrame([country]*len(df))
            
            # reformat column names
            df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
            
            # gather list of dataframes; columns are sorted for ease of merging
            df_list.append(df.reindex(sorted(df.columns), axis=1))
            
            print(list((month, country, file)))

['01', 'Denmark', 'Book1.xlsx']
['01', 'Egypt', 'Book1.xlsx']
['01', 'Ghana', 'Book1.xlsx']
['02', 'Denmark', 'Book1.xlsx']
['02', 'Egypt', 'Book1.xlsx']
['02', 'Ghana', 'Book1.xlsx']
['03', 'Denmark', 'Book1.xlsx']
['03', 'Egypt', 'Book1.xlsx']
['03', 'Ghana', 'Book1.xlsx']
['04', 'Denmark', 'Book1.xlsx']
['04', 'Egypt', 'Book1.xlsx']
['04', 'Ghana', 'Book1.xlsx']
['05', 'Denmark', 'Book1.xlsx']
['05', 'Egypt', 'Book1.xlsx']
['05', 'Ghana', 'Book1.xlsx']
['06', 'Denmark', 'Book1.xlsx']
['06', 'Egypt', 'Book1.xlsx']
['06', 'Ghana', 'Book1.xlsx']
['07', 'Denmark', 'Book1.xlsx']
['07', 'Egypt', 'Book1.xlsx']
['07', 'Ghana', 'Book1.xlsx']
['08', 'Denmark', 'Book1.xlsx']
['08', 'Egypt', 'Book1.xlsx']
['08', 'Ghana', 'Book1.xlsx']
['09', 'Denmark', 'Book1.xlsx']
['09', 'Egypt', 'Book1.xlsx']
['10', 'Denmark', 'Book1.xlsx']
['10', 'Egypt', 'Book1.xlsx']
['10', 'Ghana', 'Book1.xlsx']
['11', 'Denmark', 'Book1.xlsx']
['11', 'Egypt', 'Book1.xlsx']
['11', 'Ghana', 'Book1.xlsx']
['12', 'Denmark', 

In [15]:
data_mths = pd.concat(df_list[0:35]).reset_index(drop=True)
data_yrs = pd.concat(df_list[35:38]).reset_index(drop=True)

In [16]:
#data_mths.head(5)

In [17]:
#data_mths.info()

In [18]:
#data_yrs.head(5)

In [19]:
#data_yrs.info()

In [20]:
set(data_mths.columns) - set(data_yrs.columns)

{'del_eq_ofc', 'forward_code', 'org_eq_ofc'}

In [21]:
# merging data
full_data = data_mths.append(data_yrs, ignore_index=True).drop_duplicates(subset='b/l_no')

In [22]:
col_unique_vals = []
for i in full_data.columns:
    if isinstance(full_data[i], pd.DataFrame):
        dt = 'DF'
        nu = sum(full_data[i].nunique())
    else :
        dt = full_data[i].dtypes
        nu = full_data[i].nunique()
    ds = full_data[i].shape
    col_unique_vals.append((i, nu))
    #print("Column \'{}\' with {} unique values; \n of type {} with shape {}.".format(i, nu, dt, ds))
    print("{} COLUMN \'{}\' has [{}] unique values.".format(dt, i, nu))
    
max_nu = max([i[1] for i in col_unique_vals])
max_nu

int64 COLUMN 'activity_feu' has [70] unique values.
int64 COLUMN 'activity_teu' has [63] unique values.
object COLUMN 'also_notify_name' has [358] unique values.
object COLUMN 'b/l_no' has [5313] unique values.
object COLUMN 'booking_contact_e-mail' has [320] unique values.
object COLUMN 'booking_contact_phone_number' has [328] unique values.
object COLUMN 'booking_no' has [4891] unique values.
object COLUMN 'cmdt' has [452] unique values.
float64 COLUMN 'cmdt_code' has [455] unique values.
object COLUMN 'consignee' has [1547] unique values.
object COLUMN 'consignee_address' has [1881] unique values.
object COLUMN 'country' has [3] unique values.
object COLUMN 'customs_description' has [1323] unique values.
object COLUMN 'del' has [163] unique values.
object COLUMN 'export_name' has [441] unique values.
float64 COLUMN 'feu' has [80] unique values.
object COLUMN 'forward_name' has [710] unique values.
object COLUMN 'freight_term' has [2] unique values.
float64 COLUMN 'mea(cbm)' has [355

5313

In [23]:
# final data exploration; b/l_no has most no. of unique values
full_data.sort_values(by='b/l_no')

,activity_feu,activity_teu,also_notify_name,b/l_no,booking_contact_e-mail,booking_contact_phone_number,booking_no,cmdt,cmdt_code,consignee,consignee_address,country,customs_description,del,export_name,feu,forward_name,freight_term,mea(cbm),month,mty_p/up_cy,notify_address,notify_name,pkg,pkg_unit,pod(booking),pol(booking),por,rep_cmdt,rev.week,sailing_dt,shipper,shipper_address,si_contact_e-mail,si_contact_phone_number,teu,trunk_pod,trunk_pol,trunk_vvd,vessel_name,wgt(kgs),del_eq_ofc,forward_code,org_eq_ofc
5637,0,2,* TEL: +66 2321 7181-92\n* FAX: +66 2321 4152,AAR102100200,NaN,NaN,AAR102100200,"ANIMAL FEED, WHETHER OR NOT CONTAINING CHEMICA...",230002.0,TO ORDER OF BANGKOK BANK,"PUBLIC COMPANY LIMITED, BANGKOK",Denmark,DANISH FISHMEAL LT-999,THLCH,354 1501011 1 1334,0.0,A. HARTRODT-DKI APS\nGROENLANDSVEJ 5,P,0.0,12,NaN,"1964, 1966, 1968, 1970 ON-NUT ROAD,\nSUANLUANG...","AMERICAN MARKETING CO., LTD.",810,BG,THLCH,DKAAR,DKAAR,2300,50.0,2020-12-21 21:30:00,NORSILDMEL AS,KJERREIDVIKEN 16\n5141 FYLLINGSDALEN\nNORWAY,NaN,NaN,2.0,SGSIN,DEHAM,ALNE1545E,WES AMELIE,46100.0,NaN,NaN,NaN
5641,0,1,NaN,AAR102101200W,NaN,NaN,AAR102101200,FIBERS YARNS; NOS,550004.0,DHL GLOBAL FORWARDING (CHINA) CO NANJING BRANCH,"9/F, BLOCK 2, CITY GATEWAY\nM 359 MIDDLE JIANG...",Denmark,TEX WATER-BLOCKING GLASS,CNNKG,NaN,0.0,DHL GLOBAL FORWARDING (DENMARK) A/S\n AALBORG\...,P,0.0,12,DKAARY5,NaN,SAME AS CONSIGNEE,20,PE,CNNKG,DKAAR,DKAAR,5500,51.0,2020-12-24 08:15:00,DHL GLOBAL FORWARDING (DENMARK) A/S,MINERALVEJ 29\nDK-9000 AALBORG \nDENMARK,NaN,NaN,1.0,CNSHA,NLRTM,HJGN6314W,CERES,17538.0,NaN,NaN,NaN
5668,0,1,NaN,AAR102101300,NaN,NaN,AAR102101300,"POTATO STARCH, NOS",110803.0,TO ORDER OF DBS BANK LTD,NaN,Denmark,SUPERIOR POTATO STARCH,SGSIN,EBJ35404811,0.0,SCAN GLOBAL LOGISTICS A/S,P,0.0,12,DKAARY5,NORTH LINK BUILDING 06-70\nSINGAPORE 757695,NEWTECH MACHINERY PTE LTD\n10 ADMIRALTY STREET,810,BG,SGSIN,DKAAR,DKAAR,1100,52.0,2020-12-28 15:40:00,KMC KARTOFFELMELCENTRALEN AMBA,HERNINGVEJ 60\nDK-7330 BRANDE\nDENMARK,joso@scangl.com,0076134487,1.0,SGSIN,NLRTM,HJGN6310W,CERES,21148.0,NaN,NaN,NaN
5587,0,14,NaN,AAR102323700,NaN,NaN,AAR102323700,"ANIMAL FEED, WHETHER OR NOT CONTAINING CHEMICA...",230002.0,TO ORDER,NaN,Denmark,FF WHITE FISH MEAL,CNBEJ,354 1501003 1 1244,0.0,A. HARTRODT-DKI APS\nGROENLANDSVEJ 5,P,0.0,12,NaN,"1720RM, 17/F. JINXIMINGMEN\nBUILDIING. NO. 1 D...",FUZHOU PIN GUAN TRADE CO. LTD.,12306,BG,CNBEJ,DKAAR,DKAAR,2300,48.0,2020-12-04 13:00:00,FF SKAGEN A/S,HAVNEVAGTVEJ 5\n9990 SKAGEN\nDENMARK,NaN,NaN,14.0,CNHKG,NLRTM,ALNE1545W,WES AMELIE,312120.0,NaN,NaN,NaN
5642,5,0,** CONTAXT PERSON:\nMR. SEONG KI LEE\nTEL: +82...,AAR102389100,NaN,NaN,AAR102389100,"POTATO STARCH, NOS",110803.0,PALDO COMPANY LIMITED,577 GANGNAMDAERO SEOCHO-GU\nSOUTH KOREA\n+82-2...,Denmark,SUPERIOR POTATO STARCH,KRKWZ,EBJ35404792,5.0,SCAN GLOBAL LOGISTICS A/S\nOERESUNDSVEJ 12,P,0.0,12,DEHAMRT,DOCHEOK-MYEON 401-82 BEON-GIL\nGWANGJU-SI GYEO...,PALDO COMPANY LIMITED\n(GONGJAM DISTRIBUTION),110,BG,KRPUS,DKAAR,DKAAR,1100,51.0,2020-12-24 08:15:00,KMC KARTOFFELMELCENTRALEN A.M.B.A. HERNING VEJ 60,DK-7330 BRANDE\nDENMARK,NaN,NaN,0.0,KRPUS,NLRTM,HJGN6310E,CONDOR,110220.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2975,0,2,"JBG LOGISTICS CO.,LTD.\nJUNG-GU,PUSAN,KOREA(05...",TEM300668200,NaN,NaN,TEM300668200,"ORE, NOS",260003.0,LIANYUNGANG PANFU IMPORT & EXPORT CO. LTD.,"A02-104, HUADONG TOWN, XINGUANG \nROAD,E & T D...",Ghana,LEAD ORE,KRPUS,NaN,0.0,NaN,P,0.0,06,GHTEMY2,"C-55,2ND FLOOR, PREET VIHAR\nDELHI-110092,INDIA",SUCCESS EXIM PVT.LTD.,2,LT,KRPUS,GHTEM,GHTEM,2600,24.0,2020-06-22 08:15:00,"SUCCESS AFRICA GHANA LTD P.O. BOX AS927, VALCO...","ASHIAMAN - TEMA,GHANA",NaN,NaN,2.0,KRPUS,ESALG,HYSZ1949E,INDUSTRIAL DART,51060.0,NaN,NaN,NaN
568,0,1,HYPERO COMMODITIES F.Z.E.\nSM-OFFICE - E1 - 19...,TEM300669200,NaN,NaN,TEM300669200,"TEAKWOOD, NOS",440005.

In [39]:
full_data = full_data.sort_values(by='sailing_dt')

In [26]:
# create 'aggregate' folder for the output
aggregate_path = os.path.join(os.getcwd(), 'aggregate')

try:
    os.mkdir(aggregate_path)
except OSError:
    print("Creation of the directory %s failed" % aggregate_path)
else:
    print("Successfully created the directory %s " % aggregate_path)

Creation of the directory C:\Users\bened\Documents\UNIVERSITY\Postgraduate & Academic Opportunities\Professional Opportunities\CDGGroup\aggregate failed


In [32]:
full_data[full_data['country']=='Egypt']

(3944, 44)

In [47]:
#for i in full_data['country'].unique():
#    c_df = full_data[full_data['country']==i]
#        for j in full_data['sailing_dt'].dt.year.unique()
#            y_df = full_data[full_data['sailing_dt']==j]       

In [61]:
for i in full_data['country'].unique():
    country_path = os.path.join(aggregate_path, i)
    c_df = full_data[full_data['country']==i]
    new_path = str("aggregate" + "/" + i + "/" + "Book2.csv")
    
    try:
        os.mkdir(country_path)
    except OSError:
        print("Creation of the directory %s failed" % country_path)
    else:
        print("Successfully created the directory %s " % country_path)
    
    c_df.to_csv(new_path, index=False)

Creation of the directory C:\Users\bened\Documents\UNIVERSITY\Postgraduate & Academic Opportunities\Professional Opportunities\CDGGroup\aggregate\Egypt failed
Creation of the directory C:\Users\bened\Documents\UNIVERSITY\Postgraduate & Academic Opportunities\Professional Opportunities\CDGGroup\aggregate\Denmark failed
Creation of the directory C:\Users\bened\Documents\UNIVERSITY\Postgraduate & Academic Opportunities\Professional Opportunities\CDGGroup\aggregate\Ghana failed
